# Телеграм-бот

In [ ]:
!pip install pyTelegramBotAPI

In [ ]:
import cv2
import telebot

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
import PIL
import time

In [ ]:
# скопируем репозиторий darknet 
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# изменим кое-какие настройки
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

In [ ]:
# соберем получившееся решение, чтобы запускать файл darknet.py 
!make

In [ ]:
# скачаем файл весов модели yolov4, натренированный определять 80 классов объектов 
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V3vsIaxAlGWvK4Aar9bAiK5U0QFttKwq' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1V3vsIaxAlGWvK4Aar9bAiK5U0QFttKwq" -O yolov4-csp.weights && rm -rf /tmp/cookies.txt

In [ ]:
# импортируем darknet функции для детектирования объектов
from darknet import *
# загрузим YOLOv4 
network, class_names, class_colors = load_network("cfg/yolov4-csp.cfg", "cfg/coco.data", "yolov4-csp.weights")
width = network_width(network)
height = network_height(network)

# darknet_helper функция для детектирования объектов на изображении
def darknet_helper(img, width, height):
  darknet_image = make_image(width, height, 3)
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img_resized = cv2.resize(img_rgb, (width, height), interpolation=cv2.INTER_LINEAR)

  # пересчет координат границ объектов, чтобы можно было отобразить на кадре
  img_height, img_width, _ = img.shape
  width_ratio = img_width/width
  height_ratio = img_height/height

  # запуск модели
  copy_image_from_bytes(darknet_image, img_resized.tobytes())
  detections = detect_image(network, class_names, darknet_image)
  free_image(darknet_image)
  return detections, width_ratio, height_ratio

In [ ]:
TOKEN = "токен_вашего_бота"
bot = telebot.TeleBot(TOKEN)

In [ ]:
# хэндлер обработки команды /start, /help
@bot.message_handler(commands=["start", "help"])
def commands_start(message):
  bot.send_message(message.from_user.id, f"{message.from_user.first_name}, добрый день. Рад приветствовать вас в detected_photo_bot.\
                                             Я умею обрабатывать изображения и находить на них объекты.\
                                             Давайте попробуем обработать изображение. Загрузите фотографию и я постараюсь определить находящиеся на ней объеты.")

In [ ]:
# хэндлер обработки изображений
@bot.message_handler(content_types=['photo'])
def photographio(message):
  print(message)
  file = bot.get_file(message.photo[-1].file_id)
  bot.send_message(message.from_user.id, f'{message.from_user.first_name}, фото получил. Обрабатываю!')
  dfile = bot.download_file(file.file_path)
  fimg = message.photo[-1].file_id + '.png'

  with open(fimg, 'wb') as new_file:
     new_file.write(dfile)

  image = cv2.imread(fimg)
  detections, width_ratio, height_ratio = darknet_helper(image, width, height)

  for label, confidence, bbox in detections:
    left, top, right, bottom = bbox2points(bbox)
    left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)
    cv2.rectangle(image, (left, top), (right, bottom), class_colors[label], 2)
    cv2.putText(image, "{} [{:.2f}]".format(label, float(confidence)),
                      (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                      class_colors[label], 2)
  cv2.imwrite(fimg, image)
  with open(fimg,'rb') as fil:
        byte = fil.read()
  print('отправлено')

  bot.send_photo(message.from_user.id,  byte)
  bot.send_message(message.from_user.id, f"{message.from_user.first_name}, вот обработанное фото")

bot.infinity_polling()

In [ ]:
# пустой хэндлер для обработки неизвестных команд
@bot.message_handler()
def echo_send(message):
  bot.send_message(message.from_user.id, "Я умею только обрабатывать изображения и находить на них объекты.\nЗагрузите фото и я попробую определить, что на нём изображено")